<a href="https://colab.research.google.com/github/skngetich/msc-dsa-mod-2/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install packages

In [1]:
%matplotlib inline
from bokeh.io import output_notebook
from bokeh import models, palettes, transform
from bokeh.plotting import figure, show
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import cluster, decomposition, pipeline, preprocessing
import statsmodels
import sqlite3

Import dataset


In [2]:
from google.colab import drive
drive.mount('/content/drive')
kjkj


Mounted at /content/drive


Change Directory to the dataset folder in the colab notebooks folder

In [ ]:
%cd drive/MyDrive/'Colab Notebooks'/dataset/

print("------- LIST FILE ---------")

%ls


[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/dataset/'
/content
------- LIST FILE ---------
sample_data/


The SQLite database contains 3 tables. Survey, Question, and Answer.

- Survey (PRIMARY KEY INT `SurveyID`, TEXT Description)
- Question (PRIMARY KEY `QuestionID`, TEXT QuestionText)
- Answer (PRIMARY/FOREIGN KEY `SurveyID`, PRIMARY KEY `UserID`,  PRIMARY/FOREIGN KEY `QuestionID`, TEXT AnswerText)

SuveyID are simply survey year ie 2014, 2016, 2017, 2018, 2019.
The same question can be used for multiple surveys
Answer table is a composite table with multiple primary keys. SurveyID and QuestionID are FOREIGN KEYS.
Some questions can contain multiple answers, thus the same user can appear more than once for that questionid.

In [ ]:

# Create a SQL connection to our SQLite database
con = sqlite3.connect("mental_health.sqlite")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM answer where AnswerText > 34  limit 40 ;'):
# for row in cur.execute('pragma table_info('albums');'):
    print(row)

# Be sure to close the connection
con.close()

('37', 2014, 1, 1)
('44', 2014, 2, 1)
('35', 2014, 7, 1)
('39', 2014, 8, 1)
('42', 2014, 9, 1)
('42', 2014, 13, 1)
('36', 2014, 14, 1)
('46', 2014, 19, 1)
('36', 2014, 20, 1)
('46', 2014, 23, 1)
('41', 2014, 24, 1)
('35', 2014, 26, 1)
('35', 2014, 28, 1)
('37', 2014, 30, 1)
('42', 2014, 34, 1)
('40', 2014, 35, 1)
('38', 2014, 38, 1)
('50', 2014, 39, 1)
('35', 2014, 40, 1)
('35', 2014, 42, 1)
('38', 2014, 46, 1)
('44', 2014, 61, 1)
('35', 2014, 65, 1)
('40', 2014, 66, 1)
('36', 2014, 68, 1)
('38', 2014, 74, 1)
('38', 2014, 81, 1)
('39', 2014, 87, 1)
('40', 2014, 92, 1)
('44', 2014, 102, 1)
('35', 2014, 106, 1)
('35', 2014, 107, 1)
('45', 2014, 113, 1)
('35', 2014, 120, 1)
('36', 2014, 124, 1)
('38', 2014, 134, 1)
('39', 2014, 136, 1)
('39', 2014, 138, 1)
('38', 2014, 141, 1)
('37', 2014, 142, 1)


In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("mental_health.sqlite")

# query = """
#         SELECT question.*, answer.*
#         FROM   question 
#               LEFT JOIN answer 
#                   ON question.questionID = answer.questionID
#         UNION ALL
#         SELECT question.*, answer.*
#         FROM   answer 
#               LEFT JOIN question 
#                   ON question.questionID = answer.questionID
#         WHERE  question.questionID IS NULL
#         LIMIT 2000
# """
query = """
        SELECT *
        FROM answer
        LIMIT 1000
"""

df= pd.read_sql_query(query,con)

# Be sure to close the connection
con.close()
print(df.columns)
df.head().T
# df.values
# df2= pd.crosstab([df.SurveyID,df.UserID,df.QuestionID],df.QuestionID,df.AnswerText, aggfunc=','.join)
df2 =pd.DataFrame(data=df,index=[df.UserID,df.SurveyID,df.QuestionID],columns=df.QuestionID)
# df2 =df.set_index([ 'UserID', 'SurveyID','QuestionID'])
df2.head(120)

Index(['AnswerText', 'SurveyID', 'UserID', 'QuestionID'], dtype='object')


,,QuestionID,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
UserID,SurveyID,QuestionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,2014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
